# 檢索器策略比較

本範例展示：
1. **第1個儲存格**：載入向量資料庫
2. **第2個儲存格**：比較三種檢索策略（Similarity、MMR、Threshold）

學習目標：理解不同檢索策略的特點，根據需求選擇最適合的檢索方式

In [ ]:
# 第1個儲存格：載入向量資料庫

import os
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# 定義持久化目錄
current_dir = os.path.dirname(os.path.abspath("__file__"))
db_dir = os.path.join(current_dir, "db")
persistent_directory = os.path.join(db_dir, "chroma_db_with_metadata_chinese_nb")

# 定義嵌入模型
embeddings = HuggingFaceEmbeddings(model_name="jinaai/jina-embeddings-v2-base-zh")

# 使用嵌入函數載入現有的向量存儲
db = Chroma(persist_directory=persistent_directory,
            embedding_function=embeddings)

print("向量資料庫已載入")
print(f"資料庫位置: {persistent_directory}")

In [ ]:
# 第2個儲存格：比較三種檢索策略（Similarity、MMR、Threshold）

import os
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# 定義持久化目錄
current_dir = os.path.dirname(os.path.abspath("__file__"))
db_dir = os.path.join(current_dir, "db")
persistent_directory = os.path.join(db_dir, "chroma_db_with_metadata_chinese_nb")

# 定義嵌入模型
embeddings = HuggingFaceEmbeddings(model_name="jinaai/jina-embeddings-v2-base-zh")

# 載入向量存儲
db = Chroma(persist_directory=persistent_directory,
            embedding_function=embeddings)

# 定義使用者的問題
query = "電動機車電池如何保養？"

print("\n" + "="*70)
print(f"查詢問題：{query}")
print("="*70)

# 1. 相似度搜尋（Similarity Search）
print(f"\n{'='*70}")
print("檢索策略 1：相似度搜尋 (Similarity Search)")
print(f"{'='*70}")
print("📌 特點：返回與查詢最相似的文件")
print("⚙️  參數：k=3（返回3個結果）\n")

retriever_similarity = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)
docs_similarity = retriever_similarity.invoke(query)

for i, doc in enumerate(docs_similarity, 1):
    print(f"文件 {i}:")
    content_preview = doc.page_content[:100].replace('\n', ' ')
    print(f"{content_preview}...")
    if doc.metadata:
        print(f"來源: {doc.metadata.get('source', 'Unknown')}\n")

# 2. 最大邊際相關性（MMR - Maximal Marginal Relevance）
print(f"\n{'='*70}")
print("檢索策略 2：最大邊際相關性 (MMR)")
print(f"{'='*70}")
print("📌 特點：結果多樣化，避免重複相似的內容")
print("⚙️  參數：k=3, fetch_k=20, lambda_mult=0.5")
print("   - fetch_k: 先取20個候選文件")
print("   - lambda_mult: 0.5 平衡相關性和多樣性\n")

retriever_mmr = db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 3, "fetch_k": 20, "lambda_mult": 0.5}
)
docs_mmr = retriever_mmr.invoke(query)

for i, doc in enumerate(docs_mmr, 1):
    print(f"文件 {i}:")
    content_preview = doc.page_content[:100].replace('\n', ' ')
    print(f"{content_preview}...")
    if doc.metadata:
        print(f"來源: {doc.metadata.get('source', 'Unknown')}\n")

# 3. 相似度分數閾值（Similarity Score Threshold）
print(f"\n{'='*70}")
print("檢索策略 3：相似度分數閾值 (Similarity Score Threshold)")
print(f"{'='*70}")
print("📌 特點：只返回相似度分數超過閾值的文件")
print("⚙️  參數：score_threshold=0.1（分數 >= 0.1 才返回）\n")

retriever_threshold = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.1}
)
docs_threshold = retriever_threshold.invoke(query)

print(f"找到 {len(docs_threshold)} 個符合閾值的文件\n")
for i, doc in enumerate(docs_threshold, 1):
    print(f"文件 {i}:")
    content_preview = doc.page_content[:100].replace('\n', ' ')
    print(f"{content_preview}...")
    if doc.metadata:
        print(f"來源: {doc.metadata.get('source', 'Unknown')}\n")

# 總結比較
print(f"\n{'='*70}")
print("📊 三種策略比較總結")
print(f"{'='*70}")
print(f"""
檢索策略            返回數量
─────────────────  ────────
Similarity         {len(docs_similarity)} 個
MMR                {len(docs_mmr)} 個
Threshold          {len(docs_threshold)} 個

💡 選擇建議：
✅ Similarity：一般查詢，速度快
✅ MMR：需要多樣化結果，避免重複
✅ Threshold：品質優先，過濾低相關性結果

🔍 觀察重點：
- Similarity 和 MMR 的結果是否有差異？
- MMR 是否提供了更多元的資訊？
- Threshold 過濾掉了多少低品質結果？
""")

In [ ]:
# 第3個儲存格：整合 Chain 功能 - 比較不同檢索策略的 RAG Chain

import os
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

# 定義持久化目錄
current_dir = os.path.dirname(os.path.abspath("__file__"))
db_dir = os.path.join(current_dir, "db")
persistent_directory = os.path.join(db_dir, "chroma_db_with_metadata_chinese_nb")

# 定義嵌入模型
embeddings = HuggingFaceEmbeddings(model_name="jinaai/jina-embeddings-v2-base-zh")

# 載入向量存儲
db = Chroma(persist_directory=persistent_directory,
            embedding_function=embeddings)

# 初始化 LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 定義 Prompt 模板
prompt = ChatPromptTemplate.from_template("""你是一位專業的電動機車維護顧問，請根據提供的資料回答使用者問題。

參考資料：
{context}

問題：{question}

請提供清楚、實用的回答：""")

# 格式化文件的函數
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 使用不同檢索策略的 RAG Chain 並比較結果
def compare_retriever_rag_chains(search_type, search_kwargs, strategy_name, query):
    print(f"\n{'='*70}")
    print(f"檢索策略：{strategy_name}")
    print(f"{'='*70}")
    print(f"搜尋類型: {search_type}")
    print(f"搜尋參數: {search_kwargs}")
    print(f"{'-'*70}")
    
    # 建立檢索器
    retriever = db.as_retriever(
        search_type=search_type,
        search_kwargs=search_kwargs,
    )
    
    # 步驟 1：檢索相關文件
    print(f"\n📋 步驟 1：檢索相關文件")
    retrieved_docs = retriever.invoke(query)
    print(f"找到 {len(retrieved_docs)} 個相關文件")
    for i, doc in enumerate(retrieved_docs, 1):
        print(f"\n文件 {i} 內容預覽：")
        content_preview = doc.page_content[:120].replace('\n', ' ')
        print(f"{content_preview}...")
        if doc.metadata:
            print(f"來源: {doc.metadata.get('source', 'Unknown')}")
    
    # 步驟 2：建立 RAG Chain
    print(f"\n⛓️  步驟 2：建立 RAG Chain")
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    
    # 步驟 3：執行 Chain 並取得答案
    print(f"\n🤖 步驟 3：生成回答")
    answer = rag_chain.invoke(query)
    print(f"\n【{strategy_name} 的回答】")
    print(answer)
    print(f"\n{'-'*70}")
    
    return answer

# 定義使用者的問題
query = "電動機車電池如何保養？有哪些注意事項？"

print("\n" + "="*70)
print(f"查詢問題：{query}")
print("="*70)

# 比較不同檢索策略的 RAG Chain 效果
results = {}

# 1. 相似度搜尋（Similarity Search）
results["Similarity"] = compare_retriever_rag_chains(
    "similarity",
    {"k": 3},
    "相似度搜尋 (Similarity Search)",
    query
)

# 2. 最大邊際相關性（MMR）
results["MMR"] = compare_retriever_rag_chains(
    "mmr",
    {"k": 3, "fetch_k": 20, "lambda_mult": 0.5},
    "最大邊際相關性 (MMR)",
    query
)

# 3. 相似度分數閾值（Similarity Score Threshold）
results["Threshold"] = compare_retriever_rag_chains(
    "similarity_score_threshold",
    {"k": 3, "score_threshold": 0.1},
    "相似度分數閾值 (Similarity Score Threshold)",
    query
)

print("\n" + "="*70)
print("📊 檢索策略對 RAG Chain 的影響總結")
print("="*70)
print("""
🔍 觀察重點：
1. 檢索到的文件數量和相關性
2. 文件內容的多樣性（是否有重複資訊）
3. 最終答案的完整性和準確性

💡 策略特性比較：

📌 Similarity Search（相似度搜尋）
   ✅ 優點: 返回最相關的結果，速度快
   ❌ 缺點: 可能返回內容相似的重複結果
   🎯 適合場景: 
      - 一般查詢需求
      - 快速檢索
      - 問題明確且聚焦
   
📌 MMR（最大邊際相關性）
   ✅ 優點: 結果多樣化，涵蓋不同角度
   ❌ 缺點: 可能犧牲部分相關性，速度較慢
   🎯 適合場景:
      - 需要多樣化的答案
      - 探索性查詢
      - 想了解多個面向的資訊
      
📌 Similarity Score Threshold（相似度分數閾值）
   ✅ 優點: 保證結果品質，過濾低相關性
   ❌ 缺點: 可能返回較少結果或無結果
   🎯 適合場景:
      - 品質優先於數量
      - 精確查詢
      - 避免錯誤資訊

🎓 實戰建議：
- 一般用途 → Similarity Search
- 避免重複內容 → MMR (lambda_mult=0.5-0.7)
- 高品質要求 → Threshold (score_threshold=0.7-0.9)
- 可以組合使用：先用 MMR 確保多樣性，再用 Threshold 過濾品質

🔑 關鍵學習：
檢索策略直接影響 RAG Chain 的輸入品質！
選擇合適的策略能夠：
✅ 提供更相關的上下文
✅ 避免重複或低品質資訊
✅ 改善 LLM 生成的答案品質
""")
print("="*70)